In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import gurobipy as gp
from gurobipy import GRB
import datetime as dt
import matplotlib.pyplot as plt

In [ ]:
wl=pd.read_csv("onlywaitlists.csv")
# sum together enrollment and waitlist data
wl_data=wl.groupby(['Class']).sum().reset_index()
# dtop the row the totals waitlist number per class
wl_data=wl_data.drop(43)
print(wl_data["Cap"])
# drop excess column/rows, transpose to get groups as rows
wl_tr=wl_data.drop(['Class'], axis=1).transpose()
wl_tr=wl_tr.drop(["OTHER", "Total", "Cap"], axis=0)
# compute the total number of waitlists per group
wl_tr["total waitlists"]=wl_tr.sum(axis=1)
# total number of students in each group
wl_tr["total students"]=wl.iloc[0]

# compute the ratio betweem waitlists and total students
# this ratio can be interpeted as the average number of waitlists joined by students in this group
wl_tr["ratio"]=wl_tr["total waitlists"]/wl_tr["total students"]
# divide the number of waitlists per each course by the total number of waitlists
# this can be interpreted as the probability that a student in this group would join the waitlist for this particular class
wl_final=wl_tr[range(0,43)].div(wl_tr["total waitlists"], axis=0)
# bring back the previous columns, fill in naN values with 0
wl_final['waitlist']=wl_tr["total waitlists"]
wl_final['students']=wl_tr["total students"]
wl_final['ratio']=wl_tr["ratio"]
wl_final=wl_final.fillna(0)
# get rid of rows with no students 
wl_final = wl_final[wl_final["students"] > 2]
wl_final

In [ ]:
wl_totcap=wl.groupby(['Class']).sum()[["Total", "Cap"]]
wl_totcap['Ratio']=wl_totcap["Total"]/wl_totcap["Cap"]
wl_totcap=wl_totcap.reset_index().drop(43)
wl_totcap=wl_totcap.sort_values(by="Ratio")
wl_totcap

In [ ]:
classes=wl_totcap[32:].index.tolist()
len(classes)

In [ ]:
frac=wl_totcap[32:]["Total"].sum()/wl_totcap["Total"].sum()
frac

In [ ]:
cap=wl_totcap[32:]["Cap"].tolist()
cap

In [ ]:
mu=wl_totcap["Total"].sum()/wl_final["students"].sum()
mu

In [ ]:
prob = sp.poisson.cdf(0, mu)
prob

In [ ]:
wl_final["students"].sum()*(1-prob)*frac

In [ ]:
wl_total=wl_totcap[32:]["Total"].sum()
class_prob=wl_totcap[32:]["Total"]/wl_total

In [ ]:
def sample_class_num(mean, N):
    sample=np.round(np.random.poisson(mean, N))
    sample[sample<0]=0
    return sample

# randomly chooses courses according to the distribution
def course_ranking(class_num, course_prob):
    if(np.all(course_prob == 0)):
        print("all 0")
        return np.zeros(class_num)
    return np.random.choice(11,min(class_num, np.count_nonzero(course_prob)),p=course_prob, replace=False)

# for a certain group compute a sample of course rankings
def sample_course_ranking(size):
    class_num=np.repeat(3,size)
    rank=[course_ranking(x, class_prob) for x in class_num]
    return rank

def generate_demand_set(arr, max_bundle_size):
    possible_bundles = []
    for i in range(len(arr)):
        for j in range(i + 1, len(arr) + 1):
            bundle = arr[i:j]
            possible_bundles.append((sum(bundle), bundle))
    #put in order of utility
    possible_bundles.sort(key=lambda x: (x[0], len(x[1])), reverse=True)
    #get rid of bundles that are too large
    final_possible_bundles = [x[1] for x in possible_bundles if len(x) <= max_bundle_size]
    return final_possible_bundles

def one_hot_encode(input_list):
    encoded_vector = np.zeros(11)
    encoded_vector[input_list] = 1
    return encoded_vector.tolist()

def demand_set_to_one_hot(all_demand_set):
	one_hot_demand_sets = []
	for student_ds in all_demand_set: 
		student_one_hot_ds = []
		for bundle in student_ds: 
			one_hot = one_hot_encode(bundle)
			student_one_hot_ds.append(one_hot)
		one_hot_demand_sets.append(student_one_hot_ds)
	return one_hot_demand_sets

def get_all_demand_set(rank,max_bundle_size=3):
    all_demand_sets = []
    for student in rank:
        demand_set = generate_demand_set(student, max_bundle_size)
        all_demand_sets.append(demand_set)

    return all_demand_sets

def pad_arrays(allocations):
	max_len = len(max(allocations, key=len)) 
	for allocation in allocations: 
		while len(allocation) < max_len: 
			allocation.append(allocation[-1])
	return allocations

In [ ]:
def generate_full_student_dataset(size): 
	complete_demand_set = [] 

	rank=sample_course_ranking(size)
	all_demand_set=get_all_demand_set(rank,3)
	filtered_demand_set = []
	for ds in all_demand_set:
		if len(ds) == 0:
			continue
		filtered_demand_set.append(ds)
		
	one_hot_demand_set = demand_set_to_one_hot(filtered_demand_set)
	complete_demand_set.extend(one_hot_demand_set)

	return complete_demand_set

In [ ]:
dataset=generate_full_student_dataset()

In [ ]:
def get_prices_for_each_allocation(prices, student_demand_set):
	price_for_each_demand_set = []
	for demand_set in student_demand_set: 
		prices_np = np.array(prices)
		demand_set_np = np.array(demand_set).T
		total_price = prices_np.dot(demand_set_np)
		price_for_each_demand_set.append(total_price[0])
	
	return price_for_each_demand_set

def budget_perterbations(prices, student_demand_set, epsilon,delta, initial_budget): 
	# input: prices for each course & demand set for a student 
	# prices = [.2, .3, .1, .5] (ie, price of course at idx 0 is 0.3, price of course at idx 1 is 0.2)
	# student_demand_set = [[1,1,0,0], [1,0,0,0],[0,1,0,0]] where a student's most preferred bundle is [1,1,0,0] and they get classes 0 and 1
	# second preferred bundle is [1,0,0,0] where the student would get only class 0, etc. 

	# output: budget allocation for each student, ie if they spent 0.2 to 0.5 (noninclusive), they would want allocation [1,0,0,0]
	# if they spent 0.5 or more, they would want allocation [1,1,0,0]

	#idx 0 price is for the top ranked bundle 
	alloc=[]
	budgets=[]
	budget=initial_budget+epsilon 
	for i in student_demand_set: 
		cost=np.dot(prices, i)
		if cost>budget:
			continue
		alloc.append(i)
		budgets.append(max(cost,initial_budget-epsilon))
		while budget>initial_budget-epsilon and budget>=cost:
			budget-=delta
		if budget <= initial_budget-epsilon:
			break
	if len (alloc)==0:
		alloc.append(np.zeros(11))
		budgets.append(0)
	return (alloc, budgets)


In [ ]:
prices=np.repeat(0.1,11)
student_demand_set=dataset[0]
budget_perterbations(prices, student_demand_set, 0.2,0.01,0.3)

In [ ]:

def clearing_error_optimizer(a, c, p):
	n=len(a)
	m=len(c)
	ki=len(a[0])
	# Create a new model
	model = gp.Model("Clearing_Error_Minimization")

	# Decision variables
	# These are the dimensions of the decision variable array
	# array of binary decision variables with n rows and ki columns
	# z = model.addVars(m, lb=0.0, vtype=GRB.INTEGER, name="z")
	# x = model.addVars(n, ki, vtype=GRB.BINARY, name="x")

	# # Objective function: Minimize the l1-norm of vector z
	# model.setObjective(gp.quicksum(z[j] for j in range(m)), sense=GRB.MINIMIZE)

# clearing error variables
	z = model.addVars(m, vtype=GRB.INTEGER, name="z")
	y = model.addVar(name="y")

	x = model.addVars(n, ki, vtype=GRB.BINARY, name="x")


    # OBJECTIVE: minimize y, which will be equal to the 1-norm of z
	model.setObjective(y, sense=GRB.MINIMIZE)


    # Constraints

    #Set y == ||z||_1
	model.addGenConstrNorm(y, z, 1.0, "normconstr")
	# Constraints
	for j in range(m):
		if p[j]>0:
			model.addConstr(gp.quicksum(x[i, l] * a[i][l][j] for i in range(n) for l in range(ki)) == c[j] + z[j], f"clearing_error_positive_{j}")
		if p[j]==0:
			model.addConstr(gp.quicksum(x[i, l] * a[i][l][j] for i in range(n) for l in range(ki)) <= c[j] + z[j], f"clearing_error_nonnegative_{j}")
	
	for i in range(n):
		model.addConstr(gp.quicksum(x[i, l] for l in range(ki)) == 1, f"one_schedule_per_student_{i}")

	# Solve the model
	model.optimize()

	# Check optimization status
	if model.status == GRB.OPTIMAL:
		# Access the optimal solution
		optimal_x = model.getAttr("x", x)
		optimal_z = model.getAttr("x", z)
		optimal_obj = model.ObjVal

		# Print or use the optimal solution as needed
		print("Optimal x values:", optimal_x)
		print("Optimal z values:", optimal_z)
		return (optimal_x, optimal_z, optimal_obj)
	else:
		print(f"Optimization terminated with status {model.status}")
		# Check if the model is infeasible
		model.computeIIS()
		print("\nInfeasible constraints:")
		for c in model.getConstrs():
			if c.IISConstr:
				print(c.constrName)
		return (None, None, None)




In [ ]:
#example input
def show_classes_per_student(x,a, budgets): 
	student_classes = {}
	student_budgets={}
	for key, value in x.items(): 
		bundle = a[key[0]][key[1]]
		if value != 0: 
			if key[0] not in student_classes: 
				student_demand_set = []
				student_demand_set.append([index for index, value in enumerate(bundle) if value == 1])
				student_classes[key[0]] = student_demand_set
				budget = budgets[key[0]][key[1]]
				student_budgets[key[0]] = budget[0]

	return student_classes, student_budgets

def show_students_per_class(x, a):
	class_students = {}
	for key, value in x.items(): 
		if value == 0:
			continue
		bundle = a[key[0]][key[1]]
		for i, class_num in enumerate(bundle):
			if class_num == 1 and i not in class_students: 
				class_students[i] = [key[0]]
			elif class_num == 1: 
				class_students[i].append(key[0])
	return class_students

def get_budget_perterbations(prices, one_hot_demand_set,epsilon, delta,initial_budget):
	all_budget_perterbations = []
	all_budget_intervals = []
	for student_demand_set in one_hot_demand_set: 
		allocations, budget_intervals = budget_perterbations(prices, student_demand_set, epsilon, delta, initial_budget)
		if allocations != [] and budget_intervals != []:
			all_budget_perterbations.append(allocations)
			all_budget_intervals.append(budget_intervals)
	allocations = pad_arrays(all_budget_perterbations)
	budget_intervals = pad_arrays(all_budget_intervals)
	return allocations, budget_intervals

def pretty_print_output(x, prices,z,a,b, f):
	student_classes, student_budgets = show_classes_per_student(x,a,b)
	for key in student_classes: 
		f.write("student: "+str(key)+ " classes: "+str(student_classes[key]) +" budget: "+str(student_budgets[key])+ "\n")

	classes_per_student = show_students_per_class(x, a)
	for key in classes_per_student: 
		f.write("class: "+ str(key)+" students: "+ str(classes_per_student[key])+ "\n")
		f.write("class: " + str(key) + " capacity: " + str(len(classes_per_student[key])) + "\n")
	string_list = [str(float_value) for float_value in prices[0]]
	prices_string = ' '.join(string_list)
	f.write("prices: "+ prices_string+ "\n")
	string_list_z = [str(float_value) for float_value in z]
	prices_string_z = ' '.join(string_list_z)
	f.write("z: "+ prices_string_z+ "\n")

def aceei(delta, epsilon, prices, size): 
	f = open("out"+str(dt.datetime.now())+".txt", "w+")
	clearing_error=[]
	optimum=9999999
	i=0
	one_hot_demand_set=generate_full_student_dataset(size)
	x=0
	while optimum>=1 and i <=2000: 
		try:
			a, b = get_budget_perterbations(prices, one_hot_demand_set, epsilon,delta, 0.1)
			#print(np.array(a).shape, np.array(b).shape)	
			c = [int(i/(1200/size)) for i in cap]
			print("cap", c)
			p = prices[0]
			x, z, optimum= clearing_error_optimizer(a, c, p)
			# if i%10==0:
			# 	f.write("\n Epoch: " + str(i) + "\n")
			# 	pretty_print_output(x, prices, z.values(), a,b , f)
			print(z.values())
			prices = (np.array(prices) + np.array(list(z.values()))*delta).tolist() 
			clearing_error.append(optimum)
			i+=1
		except: 
			print("error")
			return (0,delta,size,prices, x, clearing_error)
	#f.close()
	return (1,delta,size,prices, x, clearing_error)


In [ ]:
deltas=[0.0001,0.00001,0.000005]
sizes=[50,100,150]
results=[]
for i in deltas: 
  for j in sizes: 
    res=aceei(i, 0.01, [[0] * 11], j)
    results.append(res)

In [ ]:
for i in results:
  print(i[0:3],i[5][len(i[5])-1])

In [ ]:
res=aceei(0.000005, 0.02, [[0] * 11])

In [ ]:
error = results[6][5]

## Arrange the plot

#empirical
x_axis = np.array(range(0, len(error)))
plt.plot(x_axis, error, label="Clearing error")
plt.ylabel("Clearing error (1-norm of z)")
plt.xlabel("Iteration")

#theory bound
alpha = np.sqrt(min(2*100, len(cap))* len(cap))/2
plt.axhline(alpha, color="r", label="Theory bound", linestyle="dashed")


plt.title(f"Clearing error for n={50}, m={11}, k={3}, d = {0.0001}, e = {0.01}")
plt.legend(loc="upper right")

plt.show()